In [1]:
# TODO which one?
#git clone https://github.com/lucidrains/iTransformer.git
#import iTransformer
import sys
sys.path.append('/vol/fob-vol7/nebenf21/reinbene/bene/MA/iTransformer') 
from iTransformer import iTransformer
import torch
from pathlib import Path

import torch
import torch.optim as optim
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

from utils import data_handling, helpers, training_functions
import config
import pandas as pd

window_size = 96
pred_length = (96)

print("Import succesfull")


Import succesfull


# Select dataset for transfer learning

In [25]:
# use electricity dataset
data_dict = data_handling.load_electricity()

transfer_learning_split = True

if transfer_learning_split == True:

    id_split_point = int(data_dict["train"].size(1) * 0.8)

    data_dict_source = {}
    data_dict_target = {}

    for key, value in data_dict.items():
        data_dict_source[key] = value[:,:id_split_point]
        data_dict_target[key] = value[:,id_split_point:]



    dataloader_train_source, dataloader_validation_source, dataloader_test_source = data_handling.convert_data(data_dict_source, window_size, pred_length)
    dataloader_train_target, dataloader_validation_target, dataloader_test_target = data_handling.convert_data(data_dict_target, window_size, pred_length)

else:
    dataloader_train, dataloader_validation, dataloader_test = data_handling.convert_data(data_dict, window_size, pred_length)


Feature batch shape: torch.Size([32, 96, 278])
Feature batch shape: torch.Size([32, 96, 70])


In [14]:
# eu electricity data
data_tensor = data_handling.eu_electricity_to_tensor()
data_dict, standardize_values =  data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

# convert to dataloader
dataloader_train, dataloader_validation, dataloader_test = data_handling.convert_data(data_dict, window_size, pred_length)

Feature batch shape: torch.Size([32, 96, 208])


In [2]:
# bavaria dataset
data_tensor = data_handling.load_bavaria_electricity()
data_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

# convert to datalaoder
dataloader_train, dataloader_validation, dataloader_test = data_handling.convert_data(data_dict, window_size, pred_length)

Feature batch shape: torch.Size([32, 96, 67])


# Load model that has been trained on electricity dataset

In [22]:
normalization_strategies = {#"base" : [False, False] #,
							"revin" : [True, True],
				#			"stationary" : [True, False]
}


In [30]:
for key, value in normalization_strategies.items():
    model_path = config.CONFIG_MODEL_LOCATION[key] / f"electricity_{key}_epoch_15_transfer_learning.pt"
    model_path = "/vol/fob-vol7/nebenf21/reinbene/bene/MA/outputs/models/iTransformer_baseline/electricity_tl_270_channels.pt"

    best_parameters = {'depth': 2, 'dim': 256, 'dim_head': 56, 'heads': 4, 'attn_dropout': 0.2, 'ff_mult': 4, 'ff_dropout': 0.1, 
                    'num_mem_tokens': 4, 'learning_rate': 0.0005}


    num_variates = data_dict["train"].size(1)
    num_variates = data_dict_target["train"].size(1)
    print(num_variates)

    model_config = {
        'num_variates': num_variates,
        'lookback_len': window_size,
        'depth': best_parameters["depth"],
        'dim': best_parameters["dim"],
        'num_tokens_per_variate': 1,
        'pred_length': pred_length,
        'dim_head': best_parameters["dim_head"],
        'heads': best_parameters["heads"],
        'attn_dropout': best_parameters["attn_dropout"],
        'ff_mult': best_parameters["ff_mult"],
        'ff_dropout': best_parameters["ff_dropout"],
        'num_mem_tokens': best_parameters["num_mem_tokens"],
        'use_reversible_instance_norm': value[0],
        'reversible_instance_norm_affine': value[1],
        'flash_attn': True
    }

    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")


    checkpoint = torch.load(model_path,map_location='cpu')

    # values are all set to zero (beta) and one (gamma), array needs to be adapted to num_variates
    # learned affine parameters are series specific and need to be relearned for new series
    # value are kept at 1 and 0 for stationary normalization
    if key != "base":
        checkpoint["model_state_dict"]["reversible_instance_norm.beta"] = torch.zeros(num_variates, 1, dtype=torch.float)
        checkpoint["model_state_dict"]["reversible_instance_norm.gamma"] = torch.ones(num_variates, 1, dtype=torch.float)


    model = iTransformer(**model_config).to(device)
    model.load_state_dict(checkpoint['model_state_dict'])


    # defining all needed instances
    optimizer = optim.Adam(model.parameters(), lr=best_parameters["learning_rate"])
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    writer = SummaryWriter(log_dir=config.CONFIG_LOGS_PATH["revin"])

70
Using device: cuda:1


In [36]:
a, b = next(iter(dataloader_test))
a.shape

torch.Size([32, 96, 278])

# Zero shot prediciton on the test data

In [37]:
# first we try zero shot

dataloader_test = dataloader_test_target

metrics = helpers.full_eval(model, dataloader_test, device)
metrics

for eval_metric, value in metrics[96].items():
    metrics[96][eval_metric] = value.item()

metrics_df = pd.DataFrame.from_dict(metrics[96], orient='index')
metrics_df.rename(columns={0: "metrics"}, inplace=True)

metrics_df.to_csv(f"{config.CONFIG_OUTPUT_PATH['iTransformer_baseline']}/transfer.csv")

metrics

Epoch: Validating: 100%|██████████| 86/86 [00:01<00:00, 55.74it/s]


{96: {'mse': 0.17870073020458221,
  'mae': 0.2555026710033417,
  'p10': 0.027185220271348953,
  'p50': 0.15552885830402374,
  'p90': 0.5825834274291992},
 192: {'mse': 0, 'mae': 0, 'p10': 0, 'p50': 0, 'p90': 0},
 336: {'mse': 0, 'mae': 0, 'p10': 0, 'p50': 0, 'p90': 0},
 720: {'mse': 0, 'mae': 0, 'p10': 0, 'p50': 0, 'p90': 0}}

# Fine-tune pretrained model on small subset of training dataset and predict on the same test dataset

In [21]:
# now we train on a small subset of the training data for multiple epochs and evaluate again

epoch = 1

for epoch in range(1, epoch + 1):
    training_functions.train_one_epoch(epoch, model, device, dataloader_train, dataloader_validation, optimizer, scheduler, writer)

metrics = helpers.full_eval(model, dataloader_test, device)
metrics

for eval_metric, value in metrics[96].items():
    metrics[96][eval_metric] = value.item()

metrics_df = pd.DataFrame.from_dict(metrics[96], orient='index')
metrics_df.rename(columns={0: key}, inplace=True)

#metrics_df.to_csv(f"{config.CONFIG_OUTPUT_PATH['iTransformer_baseline']}/metrics_{key}_epochs{epoch}_revin.csv")

metrics

Epoch: 1: 100%|██████████| 948/948 [00:27<00:00, 34.14it/s]


Epoch 1, MSE-Loss: 0.09739486943883231, LR: 0.0005


Epoch: Validating: 100%|██████████| 131/131 [00:01<00:00, 103.51it/s]


Validation metrics: {96: {'mse': tensor(0.6840, device='cuda:1')}, 192: {'mse': 0}, 336: {'mse': 0}, 720: {'mse': 0}}


Epoch: Validating: 100%|██████████| 267/267 [00:03<00:00, 86.97it/s]


{96: {'mse': 1.006528615951538,
  'mae': 0.5427948236465454,
  'p10': 0.0423695333302021,
  'p50': 0.2917636036872864,
  'p90': 1.3048157691955566},
 192: {'mse': 0, 'mae': 0, 'p10': 0, 'p50': 0, 'p90': 0},
 336: {'mse': 0, 'mae': 0, 'p10': 0, 'p50': 0, 'p90': 0},
 720: {'mse': 0, 'mae': 0, 'p10': 0, 'p50': 0, 'p90': 0}}

In [ ]:
# we load the pretrained model again and do full fine tuning on the target dataset

best_parameters = {'depth': 2, 'dim': 256, 'dim_head': 56, 'heads': 4, 'attn_dropout': 0.2, 'ff_mult': 4, 'ff_dropout': 0.1, 
                   'num_mem_tokens': 4, 'learning_rate': 0.0005}


model_config = {
    'num_variates': data_dict["train"].size(1),
    'lookback_len': window_size,
    'depth': best_parameters["depth"],
    'dim': best_parameters["dim"],
    'num_tokens_per_variate': 1,
    'pred_length': pred_length,
    'dim_head': best_parameters["dim_head"],
    'heads': best_parameters["heads"],
    'attn_dropout': best_parameters["attn_dropout"],
    'ff_mult': best_parameters["ff_mult"],
    'ff_dropout': best_parameters["ff_dropout"],
    'num_mem_tokens': best_parameters["num_mem_tokens"],
    'use_reversible_instance_norm': False,
    'reversible_instance_norm_affine': False,
    'flash_attn': True
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model_path = config.CONFIG_MODEL_LOCATION[normalization_strategie]
checkpoint = torch.load(model_path)
model = iTransformer(**model_config).to(device)
model.load_state_dict(checkpoint['model_state_dict'])


# defining all needed instances
optimizer = optim.Adam(model.parameters(), lr=best_parameters["learning_rate"])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
writer = SummaryWriter(log_dir=config.CONFIG_LOGS_PATH[normalization_strategie])

# Fine-tuning with the full bavaria dataset and prediciton on the same test dataset

In [ ]:
# bavaria dataset
data_tensor = data_handling.load_bavaria_electricity()
data_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

# convert to datalaoder
dataloader_train, dataloader_validation, dataloader_test = data_handling.convert_data(data_dict, window_size, pred_length)

In [ ]:
# now we train on a small subset of the training data for multiple epochs and evaluate again

epoch = 10
training_functions.train_one_epoch(epoch, model, device, dataloader_train, dataloader_validation, optimizer, scheduler, writer)

metrics = training_functions.full_eval(model, dataloader_test)
metrics

for eval_metric, value in metrics[96].items():
    metrics[96][eval_metric] = value.item()

metrics_df = pd.DataFrame.from_dict(metrics[96], orient='index')
metrics_df.rename(columns={0: key}, inplace=True)

metrics_df.to_csv(f"{config.CONFIG_OUTPUT_PATH['iTransformer_baseline']}/metrics_{key}_epochs{epoch}_revin.csv")
